# **User 관심도 점수를 카드 혜택에 맞춰 카드별 매칭 점수 도출**
# 1. 필요한 라이브러리 및 데이터 불러오기

In [124]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [125]:
import pandas as pd
card = pd.read_csv('/content/drive/MyDrive/CARD-LANGCHAIN/data/card_data.csv')
interests = pd.read_csv('/content/drive/MyDrive/CARD-LANGCHAIN/data/관심도모델링1차데이터.csv')

# 2. 카드 혜택 대분류 카테고리 뽑기

In [126]:
card.head()

,Card Name,Annual_fee,Base Record,Category_List
0,하나 트래블로그 SKYPASS 신용카드,"국내 4만8천원, 해외 4만8천원, 가족겸용 없음",조건없음,['항공마일리지']
1,DA카드의정석Ⅱ,"국내 1만2천원, 해외 1만2천원",직전 1개월 합계 30만원 이상,"['쇼핑', '레저', '카페/베이커리', '편의점', '대형마트', '의료', '뷰티', '대중교통', '외식']"
2,삼성카드 & MILEAGE PLATINUM(스카이패스),"국내 4만7천원, 해외 4만9천원",조건없음,"['쇼핑', '주유', '카페/베이커리', '편의점', '대중교통', '간편결제', '대형마트', '외식', '통신', '교육', '의료', '레저', '육아', '문화', '반려동물', '뷰티', '영화', '항공마일리지']"
3,현대카드ZERO Edition3(할인형),"국내 1만5천원, 해외 1만5천원, 가족 5천원",조건없음,"['대중교통', '간편결제', '편의점', '통신', '카페/베이커리', '대형마트', '주유', '외식', '쇼핑', '영화', '뷰티', '레저', '문화', '교육', '육아', '의료', '반려동물', '하이패스']"
4,삼성카드 taptap O,"국내 1만원, 해외 1만원",직전 1개월 합계 30만원 이상,"['포인트/캐시백', '카페/베이커리', '쇼핑', '통신', '대중교통', '영화', '편의점']"


In [127]:
import ast
card['Category_List'] = card['Category_List'].apply(ast.literal_eval)

In [128]:
category_dict = {
'항공' : '항공마일리지',
'쇼핑/간편결제' : ['쇼핑', '간편결제' , '포인트/캐시백', '편의점', '대형마트'],
'카페/베이커리' :['카페', '베이커리', '외식'],
'의료' : '의료'
,'반려동물': '반려동물'
,'뷰티': '뷰티'
,'교통/주유' : ['대중교통', '주유', '하이패스'],
'교육/육아': ['교육', '육아'],
'문화': ['문화', '레저', '영화'],
'납부': ['통신', '관리비', '납부 혜택']}

# 매핑 함수
def map_category_list_to_key(category_list):
  keys = set()
  for category in category_list:
    for key, values in category_dict.items():
        if category in values:
            keys.add(key)
  return list(keys)

# 대분류 카테고리 컬럼 추가
card['category'] = card['Category_List'].apply(map_category_list_to_key)

card.head()

,Card Name,Annual_fee,Base Record,Category_List,category
0,하나 트래블로그 SKYPASS 신용카드,"국내 4만8천원, 해외 4만8천원, 가족겸용 없음",조건없음,[항공마일리지],[항공]
1,DA카드의정석Ⅱ,"국내 1만2천원, 해외 1만2천원",직전 1개월 합계 30만원 이상,"[쇼핑, 레저, 카페/베이커리, 편의점, 대형마트, 의료, 뷰티, 대중교통, 외식]","[뷰티, 카페/베이커리, 쇼핑/간편결제, 의료, 교통/주유, 문화]"
2,삼성카드 & MILEAGE PLATINUM(스카이패스),"국내 4만7천원, 해외 4만9천원",조건없음,"[쇼핑, 주유, 카페/베이커리, 편의점, 대중교통, 간편결제, 대형마트, 외식, 통신, 교육, 의료, 레저, 육아, 문화, 반려동물, 뷰티, 영화, 항공마일리지]","[반려동물, 뷰티, 납부, 카페/베이커리, 쇼핑/간편결제, 의료, 교통/주유, 항공, 문화, 교육/육아]"
3,현대카드ZERO Edition3(할인형),"국내 1만5천원, 해외 1만5천원, 가족 5천원",조건없음,"[대중교통, 간편결제, 편의점, 통신, 카페/베이커리, 대형마트, 주유, 외식, 쇼핑, 영화, 뷰티, 레저, 문화, 교육, 육아, 의료, 반려동물, 하이패스]","[반려동물, 뷰티, 납부, 카페/베이커리, 쇼핑/간편결제, 의료, 교통/주유, 문화, 교육/육아]"
4,삼성카드 taptap O,"국내 1만원, 해외 1만원",직전 1개월 합계 30만원 이상,"[포인트/캐시백, 카페/베이커리, 쇼핑, 통신, 대중교통, 영화, 편의점]","[쇼핑/간편결제, 납부, 문화, 교통/주유]"


# 3. 혜택 별 카드 매칭

In [129]:
card['category']

,category
0,[항공]
1,"[뷰티, 카페/베이커리, 쇼핑/간편결제, 의료, 교통/주유, 문화]"
2,"[반려동물, 뷰티, 납부, 카페/베이커리, 쇼핑/간편결제, 의료, 교통/주유, 항공, 문화, 교육/육아]"
3,"[반려동물, 뷰티, 납부, 카페/베이커리, 쇼핑/간편결제, 의료, 교통/주유, 문화, 교육/육아]"
4,"[쇼핑/간편결제, 납부, 문화, 교통/주유]"
...,...
161,"[뷰티, 납부, 카페/베이커리, 쇼핑/간편결제, 의료, 교통/주유, 문화, 교육/육아]"
162,[항공]
163,"[쇼핑/간편결제, 카페/베이커리, 문화]"
164,"[쇼핑/간편결제, 교통/주유]"


In [130]:
interests.head()

,user_id,category,암묵적관심도,명시적관심도
0,1,교육/육아,1,0
1,1,문화,0,1
2,1,납부,0,1
3,1,항공,0,1
4,1,반려동물,1,0


user_id별로 관심 카테고리가 있는 카드 혜택을 매칭시켜
암묵적 관심도는 *0.7, 명시적관심도 * 0.9 를 곱해
매칭 점수가 가장 큰 카드뽑기

In [131]:
def map_category_list_to_key(category_list):
  keys = set()
  for category in category_list:
    for key, values in category_dict.items():
        if category in values:
            keys.add(key)
  return list(keys)

In [132]:
card

,Card Name,Annual_fee,Base Record,Category_List,category
0,하나 트래블로그 SKYPASS 신용카드,"국내 4만8천원, 해외 4만8천원, 가족겸용 없음",조건없음,[항공마일리지],[항공]
1,DA카드의정석Ⅱ,"국내 1만2천원, 해외 1만2천원",직전 1개월 합계 30만원 이상,"[쇼핑, 레저, 카페/베이커리, 편의점, 대형마트, 의료, 뷰티, 대중교통, 외식]","[뷰티, 카페/베이커리, 쇼핑/간편결제, 의료, 교통/주유, 문화]"
2,삼성카드 & MILEAGE PLATINUM(스카이패스),"국내 4만7천원, 해외 4만9천원",조건없음,"[쇼핑, 주유, 카페/베이커리, 편의점, 대중교통, 간편결제, 대형마트, 외식, 통신, 교육, 의료, 레저, 육아, 문화, 반려동물, 뷰티, 영화, 항공마일리지]","[반려동물, 뷰티, 납부, 카페/베이커리, 쇼핑/간편결제, 의료, 교통/주유, 항공, 문화, 교육/육아]"
3,현대카드ZERO Edition3(할인형),"국내 1만5천원, 해외 1만5천원, 가족 5천원",조건없음,"[대중교통, 간편결제, 편의점, 통신, 카페/베이커리, 대형마트, 주유, 외식, 쇼핑, 영화, 뷰티, 레저, 문화, 교육, 육아, 의료, 반려동물, 하이패스]","[반려동물, 뷰티, 납부, 카페/베이커리, 쇼핑/간편결제, 의료, 교통/주유, 문화, 교육/육아]"
4,삼성카드 taptap O,"국내 1만원, 해외 1만원",직전 1개월 합계 30만원 이상,"[포인트/캐시백, 카페/베이커리, 쇼핑, 통신, 대중교통, 영화, 편의점]","[쇼핑/간편결제, 납부, 문화, 교통/주유]"
...,...,...,...,...,...
161,LOCA PLATINUM(할인형),"국내 10만원, 해외 10만원, 가족겸용 없음",조건없음,"[카페/베이커리, 쇼핑, 뷰티, 대형마트, 편의점, 주유, 외식, 영화, 통신, 교육, 육아, 레저, 문화, 의료, 간편결제]","[뷰티, 납부, 카페/베이커리, 쇼핑/간편결제, 의료, 교통/주유, 문화, 교육/육아]"
162,I-MILEAGE(아시아나),"국내 4만1천원, 해외 4만2천원",직전 1개월 합계 50만원 이상,"[항공마일리지, 카페/베이커리]",[항공]
163,The CJ-현대카드M Edition2,"국내 1만5천원, 해외 2만원",조건없음,"[쇼핑, 영화, 외식, 카페/베이커리, 포인트/캐시백]","[쇼핑/간편결제, 카페/베이커리, 문화]"
164,KB국민 그린카드(서울형),"국내전용 없음, 해외겸용 없음",직전 1개월 합계 20만원 이상,"[포인트/캐시백, 대중교통, 쇼핑, 대형마트]","[쇼핑/간편결제, 교통/주유]"


In [133]:
card[
    'category'
][3][1]

'뷰티'

In [134]:
interests.head()

,user_id,category,암묵적관심도,명시적관심도
0,1,교육/육아,1,0
1,1,문화,0,1
2,1,납부,0,1
3,1,항공,0,1
4,1,반려동물,1,0


In [135]:
card.head()

,Card Name,Annual_fee,Base Record,Category_List,category
0,하나 트래블로그 SKYPASS 신용카드,"국내 4만8천원, 해외 4만8천원, 가족겸용 없음",조건없음,[항공마일리지],[항공]
1,DA카드의정석Ⅱ,"국내 1만2천원, 해외 1만2천원",직전 1개월 합계 30만원 이상,"[쇼핑, 레저, 카페/베이커리, 편의점, 대형마트, 의료, 뷰티, 대중교통, 외식]","[뷰티, 카페/베이커리, 쇼핑/간편결제, 의료, 교통/주유, 문화]"
2,삼성카드 & MILEAGE PLATINUM(스카이패스),"국내 4만7천원, 해외 4만9천원",조건없음,"[쇼핑, 주유, 카페/베이커리, 편의점, 대중교통, 간편결제, 대형마트, 외식, 통신, 교육, 의료, 레저, 육아, 문화, 반려동물, 뷰티, 영화, 항공마일리지]","[반려동물, 뷰티, 납부, 카페/베이커리, 쇼핑/간편결제, 의료, 교통/주유, 항공, 문화, 교육/육아]"
3,현대카드ZERO Edition3(할인형),"국내 1만5천원, 해외 1만5천원, 가족 5천원",조건없음,"[대중교통, 간편결제, 편의점, 통신, 카페/베이커리, 대형마트, 주유, 외식, 쇼핑, 영화, 뷰티, 레저, 문화, 교육, 육아, 의료, 반려동물, 하이패스]","[반려동물, 뷰티, 납부, 카페/베이커리, 쇼핑/간편결제, 의료, 교통/주유, 문화, 교육/육아]"
4,삼성카드 taptap O,"국내 1만원, 해외 1만원",직전 1개월 합계 30만원 이상,"[포인트/캐시백, 카페/베이커리, 쇼핑, 통신, 대중교통, 영화, 편의점]","[쇼핑/간편결제, 납부, 문화, 교통/주유]"


In [136]:

matching_card= []
for i, card_row in card.iterrows():
  total_score = 0
  for idx, user_row in interests.iterrows():
    score = 0
    # 카드의 혜택 카테고리와 사용자의 관심 카테고리 비교
    if user_row['category'] in card_row['category']:

      if user_row['암묵적관심도'] >= 1:
        score += 0.7 * user_row['암묵적관심도']
      if user_row['명시적관심도'] == 1:
        score += 0.9

        # 매칭 데이터 기록
    matching_card.append({
            'user_id': user_row['user_id'],
            'category': user_row['category'],
            'Card_category': card_row['category'],
            'Card Name': card_row['Card Name'],
            'matching_score': score
        })

matching_card_df = pd.DataFrame(matching_card)

# 카드별 총 점수 계산
final_scores = matching_card_df.groupby(['user_id','Card Name']).agg({
    'matching_score': 'sum'
}).reset_index()

final_scores

,user_id,Card Name,matching_score
0,1,ALL 우리카드 Infinite,0.7
1,1,ALL 우리카드 Premium,1.6
2,1,BC 바로 K-패스 카드,2.5
3,1,BC 바로 MACAO 카드,0.7
4,1,BC 바로 리워드 플러스,3.2
...,...,...,...
1655,10,현대카드Z play,1.6
1656,10,현대카드Z work Edition2,2.3
1657,10,현대카드ZERO Edition3(포인트형),4.6
1658,10,현대카드ZERO Edition3(할인형),4.6


In [137]:
matching_card_df


,user_id,category,Card_category,Card Name,matching_score
0,1,교육/육아,[항공],하나 트래블로그 SKYPASS 신용카드,0.0
1,1,문화,[항공],하나 트래블로그 SKYPASS 신용카드,0.0
2,1,납부,[항공],하나 트래블로그 SKYPASS 신용카드,0.0
3,1,항공,[항공],하나 트래블로그 SKYPASS 신용카드,0.9
4,1,반려동물,[항공],하나 트래블로그 SKYPASS 신용카드,0.0
...,...,...,...,...,...
8129,10,반려동물,"[쇼핑/간편결제, 문화, 교통/주유]",KB국민 Green Wave 1.5℃ 카드,0.0
8130,10,항공,"[쇼핑/간편결제, 문화, 교통/주유]",KB국민 Green Wave 1.5℃ 카드,0.0
8131,10,교육/육아,"[쇼핑/간편결제, 문화, 교통/주유]",KB국민 Green Wave 1.5℃ 카드,0.0
8132,10,쇼핑/간편결제,"[쇼핑/간편결제, 문화, 교통/주유]",KB국민 Green Wave 1.5℃ 카드,0.7


In [153]:
final_scores[final_scores['user_id'] == 6].sort_values('matching_score')

,user_id,Card Name,matching_score
844,6,I-MILEAGE(대한항공),0.0
964,6,카드의정석 EVERY MILE SKYPASS,0.0
845,6,I-MILEAGE(아시아나),0.0
955,6,아시아나 신한카드 Air 1.5,0.0
972,6,하나 트래블로그 SKYPASS 신용카드,0.0
...,...,...,...
900,6,디지로카 Las Vegas,6.2
941,6,신한카드 Simple+,6.2
905,6,롯데 국민행복카드,6.2
965,6,카드의정석 EVERY POINT,6.2


In [139]:
final_scores[(final_scores['user_id'] == 1) & (final_scores['Card Name'] == '현대카드M')]

,user_id,Card Name,matching_score
155,1,현대카드M,3.9


In [155]:
card[card['Card Name']=='KB국민 와이즈카드']['category']

,category
46,"[뷰티, 납부, 카페/베이커리, 쇼핑/간편결제, 의료, 교통/주유, 문화, 교육/육아]"


In [141]:
interests[interests['user_id']==6]

,user_id,category,암묵적관심도,명시적관심도
22,6,뷰티,1,1
23,6,쇼핑/간편결제,2,0
24,6,교통/주유,1,1
25,6,교육/육아,1,0
26,6,납부,0,1


In [142]:
pd.set_option('display.max_colwidth', None)

# 4. 연회비

In [143]:
# 국내 금액 추출 함수
def extract_domestic_fee(fee_string):
    import re
    # 국내 금액만 추출
    match = re.search(r'국내 (?:(\d+)만)?(\d*)천?원?', fee_string)
    if match:
        # '만'과 '천' 단위 합산하여 숫자로 변환
        ten_thousand = int(match.group(1)) * 10000 if match.group(1) else 0
        thousand = int(match.group(2)) * 1000 if match.group(2) else 0
        return ten_thousand + thousand
    return 0  # 국내 금액이 없으면 0 반환

# 데이터프레임의 Annual_fee 열에서 국내 금액 추출
card['Domestic_Fee'] = card['Annual_fee'].apply(extract_domestic_fee)


# 5. 카드 혜택 매칭 점수와 연회비 함께 본 후, 최종 카드 선정

In [144]:
final_scores

,user_id,Card Name,matching_score
0,1,ALL 우리카드 Infinite,0.7
1,1,ALL 우리카드 Premium,1.6
2,1,BC 바로 K-패스 카드,2.5
3,1,BC 바로 MACAO 카드,0.7
4,1,BC 바로 리워드 플러스,3.2
...,...,...,...
1655,10,현대카드Z play,1.6
1656,10,현대카드Z work Edition2,2.3
1657,10,현대카드ZERO Edition3(포인트형),4.6
1658,10,현대카드ZERO Edition3(할인형),4.6


In [145]:
data = pd.merge(card, final_scores, how='right', on='Card Name')

In [146]:
sorted_data = data.groupby(['user_id','Card Name'])[['matching_score','Domestic_Fee']].sum().sort_values(by=['user_id','matching_score','Domestic_Fee'],ascending=[True,False,False])

In [147]:
sorted_data

matching_score  Domestic_Fee
user_id Card Name                                                   
1       삼성카드 & MILEAGE PLATINUM(스카이패스)             4.8         47000
        the Pink Edition2                          3.9        150000
        현대카드MM                                     3.9         70000
        현대카드X                                      3.9         50000
        현대카드M                                      3.9         30000
...                                                ...           ...
10      현대카드X HYBRID                               0.7          2000
        현대백화점 FIT카드                                0.7             0
        현대백화점 해피포인트카드                              0.7             0
        현대백화점 현대오일뱅크카드                             0.7             0
        현대백화점카드                                    0.7             0

[1660 rows x 2 columns]

In [151]:
sorted_data.reset_index().sort_values(['user_id','matching_score','Domestic_Fee'],ascending=[True, False, True]).groupby('user_id').head(3)

,user_id,Card Name,matching_score,Domestic_Fee
0,1,삼성카드 & MILEAGE PLATINUM(스카이패스),4.8,47000
8,1,삼성 iD SIMPLE 카드,3.9,7000
7,1,삼성카드 taptap S,3.9,10000
174,2,삼성 iD SIMPLE 카드,4.8,7000
173,2,삼성카드 taptap S,4.8,10000
171,2,현대카드ZERO Edition3(포인트형),4.8,15000
340,3,삼성 iD SIMPLE 카드,6.2,7000
339,3,삼성카드 taptap S,6.2,10000
337,3,현대카드ZERO Edition3(포인트형),6.2,15000
506,4,삼성 iD SIMPLE 카드,4.1,7000


In [156]:
card['Annual_fee'].loc[card['Card Name'] == '삼성카드 & MILEAGE PLATINUM(스카이패스)']

,Annual_fee
2,"국내 4만7천원, 해외 4만9천원"
